In [1]:
import pandas as pd
import geopandas
from folium import Circle, Marker, Icon, Map
from pymongo import MongoClient, GEOSPHERE
conn = MongoClient("localhost:27017")
db = conn.get_database("madrid")
import random

In [2]:
df_restaurants = pd.read_csv("data/restaurants.csv", index_col=0)
df_snacks = pd.read_csv("data/snacks.csv", index_col=0)
df_culture = pd.read_csv("data/culture.csv", index_col=0)
df_party = pd.read_csv("data/party.csv", index_col=0)
df_drinks = pd.read_csv("data/drinks.csv", index_col=0)
df_outdoors = pd.read_csv("data/outdoors.csv", index_col=0)
df_leisure = pd.read_csv("data/leisure.csv", index_col=0)

In [3]:
#list(df_restaurants.CP.unique())

In [15]:
leisure = db.get_collection("leisure")
outdoors = db.get_collection("outdoors")
restaurants = db.get_collection("restaurants")
drinks = db.get_collection("drinks")
party = db.get_collection("party")


## Team-building

In [16]:
db.list_collection_names()

['restaurants',
 'culture',
 'snacks',
 'todo',
 'leisure',
 'outdoors',
 'transport',
 'drinks',
 'party']

In [17]:
intersection = set(list(df_leisure.CP.unique())).intersection(set(list(df_restaurants.CP.unique())))

In [18]:
cp = list(intersection)

In [19]:
cp_random = random.choice(cp)

In [20]:
leis = list(leisure.find({"CP":str(int(cp_random))}))

In [21]:
rest = list(restaurants.find({"CP":str(int(cp_random))}))

In [22]:
l_random = random.choice(leis)
r_random = random.choice(rest)

In [23]:
l_random

{'_id': ObjectId('609ff458b70b40a97f6e74ef'),
 'name': 'The Box - Escape Room',
 'rating': 4.7,
 'address': 'Calle de Covarrubias, 22, 28010 Madrid, Spain',
 'price_level': 2.0,
 'place': 'escape room',
 'CP': '28010',
 'latitude': 40.4300619,
 'longitude': -3.6975783,
 'geometry': {'type': 'Point', 'coordinates': [-3.6975783, 40.4300619]},
 'price': '15-25€'}

In [24]:
r_random

{'_id': ObjectId('609ff4562ae6282c52dde571'),
 'name': 'Canalla Lounge',
 'rating': 4.2,
 'address': 'Calle de García de Paredes, 2, 28010 Madrid, Spain',
 'price_level': 2.0,
 'place': 'mexican',
 'CP': '28010',
 'latitude': 40.436983,
 'longitude': -3.703971,
 'geometry': {'type': 'Point', 'coordinates': [-3.703971, 40.436983]},
 'price': '15-25€'}

## Friends

In [141]:
intersection = set(list(df_drinks.CP.unique())).intersection(set(list(df_party.CP.unique())))

cp = list(intersection)

cp_random = random.choice(cp)

drinks_ = list(drinks.find({"CP":str(int(cp_random))}))

party_ = list(party.find({"CP":str(int(cp_random))}))

d_random = random.choice(drinks_)
p_random = random.choice(party_)

In [142]:
d_random

{'_id': ObjectId('609ff456b948afe42d69db20'),
 'name': 'Brewery Norwich International',
 'rating': 4.4,
 'address': 'Calle de la Marroquina, 38, 28030 Madrid, Spain',
 'price_level': 1.0,
 'place': 'brewery',
 'CP': '28030',
 'latitude': 40.407804,
 'longitude': -3.64773,
 'geometry': {'type': 'Point', 'coordinates': [-3.64773, 40.407804]},
 'price': '15-25€'}

In [133]:
p_random.pop("geometry")

{'type': 'Point', 'coordinates': [-3.678826, 40.4328007]}

In [143]:
p_random

{'_id': ObjectId('609ff457760891fde7e5fc9b'),
 'name': 'Pub Shark',
 'rating': 4.4,
 'address': 'Calle del Corregidor Diego de Valderrábano, 1, 28030 Madrid, Spain',
 'price_level': 1.0,
 'place': 'pub',
 'CP': '28030',
 'latitude': 40.4084733,
 'longitude': -3.6621067,
 'geometry': {'type': 'Point', 'coordinates': [-3.6621067, 40.4084733]},
 'price': '15-25€'}

In [144]:
df1 = pd.DataFrame.from_dict(d_random)
df2 = pd.DataFrame.from_dict(p_random)
df = pd.concat([df1,df2])

In [145]:
df

,_id,name,rating,address,price_level,place,CP,latitude,longitude,geometry,price
coordinates,609ff456b948afe42d69db20,Brewery Norwich International,4.4,"Calle de la Marroquina, 38, 28030 Madrid, Spain",1.0,brewery,28030,40.407804,-3.647730,"[-3.64773, 40.407804]",15-25€
type,609ff456b948afe42d69db20,Brewery Norwich International,4.4,"Calle de la Marroquina, 38, 28030 Madrid, Spain",1.0,brewery,28030,40.407804,-3.647730,Point,15-25€
coordinates,609ff457760891fde7e5fc9b,Pub Shark,4.4,"Calle del Corregidor Diego de Valderrábano, 1,...",1.0,pub,28030,40.408473,-3.662107,"[-3.6621067, 40.4084733]",15-25€
type,609ff457760891fde7e5fc9b,Pub Shark,4.4,"Calle del Corregidor Diego de Valderrábano, 1,...",1.0,pub,28030,40.408473,-3.662107,Point,15-25€


## General

In [182]:
def planes():

    #df_1 = "df_" + plan_1
    #df_2 = "df_" + plan_2

   # intersection = set(list(locals()[df_1].CP.unique())).intersection(set(list(locals()[df_2].CP.unique())))
    intersection = set(list(df_restaurants.CP.unique())).intersection(set(list(df_outdoors.CP.unique())))

    cp = list(intersection)

    cp_random = random.choice(cp)

    plan1 = list(restaurants.find({"CP":str(int(cp_random))}))

    plan2 = list(outdoors.find({"CP":str(int(cp_random))}))

    p1_random = random.choice(plan1)
    p2_random = random.choice(plan2)

    df1 = pd.DataFrame.from_dict(p1_random)
    df2 = pd.DataFrame.from_dict(p2_random)
    df = pd.concat([df1,df2])

    return df

In [183]:
planes()

,_id,name,rating,address,price_level,place,CP,latitude,longitude,geometry,price
coordinates,609ff4562ae6282c52dde4d8,The Quick Greek Alcalá de Henares,4.1,"Av. Guadalajara, 3, 28805 Alcalá de Henares, M...",1.0,greek,28805,40.485838,-3.360577,"[-3.3605771, 40.4858378]",1-15€
type,609ff4562ae6282c52dde4d8,The Quick Greek Alcalá de Henares,4.1,"Av. Guadalajara, 3, 28805 Alcalá de Henares, M...",1.0,greek,28805,40.485838,-3.360577,Point,1-15€
coordinates,609ff45712159f0ceb2a424c,Encin Golf Hotel,4.1,"A-2 Km 35 600, 28805 Alcalá de Henares, Spain",1.0,golf,28805,40.522474,-3.312070,"[-3.3120705, 40.5224737]",15-25€
type,609ff45712159f0ceb2a424c,Encin Golf Hotel,4.1,"A-2 Km 35 600, 28805 Alcalá de Henares, Spain",1.0,golf,28805,40.522474,-3.312070,Point,15-25€
